In [1]:
import tensorflow as tf
import utils
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# load training data and labels
data0 = utils.load_h5("ply_data_train0.h5")
data1 = utils.load_h5("ply_data_train1.h5")
data2 = utils.load_h5("ply_data_train2.h5")
data3 = utils.load_h5("ply_data_train3.h5")
data4 = utils.load_h5("ply_data_train4.h5")

train_data = data0[0]
print(np.shape(train_data))

train_labels = data0[1]
catagory_names = utils.get_category_names()
print(np.shape(train_labels))

(2048, 2048, 3)
(2048, 1)


In [13]:
# aggregate training data, training label
# train_data = np.append(data0[0], data1[0], axis=0)
# train_data = np.append(train_data, data2[0], axis=0)
# train_data = np.append(train_data, data3[0], axis=0)
# train_data = np.append(train_data, data4[0], axis=0)
# print(np.shape(train_data))

# train_labels = np.append(data0[1], data1[1], axis=0)
# train_labels = np.append(train_labels, data2[1], axis=0)
# train_labels = np.append(train_labels, data3[1], axis=0)
# train_labels = np.append(train_labels, data4[1], axis=0)
# print(np.shape(train_labels))

(9840, 2048, 3)
(9840, 1)


In [4]:
# load test data
test0 = utils.load_h5("ply_data_test0.h5")
test1 = utils.load_h5("ply_data_test1.h5")
# print(np.shape(test1[0]))

(420, 2048, 3)


In [5]:
# aggregate test data, test label
test_data = np.append(test0[0], test1[0], axis=0)
test_labels = np.append(test0[1], test1[1], axis=0)
print(np.shape(test_data))
print(np.shape(test_labels))

(2468, 2048, 3)
(2468, 1)


In [7]:
# one hot encode train_labels
train_labels = []
for l in data0[1]:
    one_hot = np.zeros(40, dtype=np.int)
    one_hot[l[0]] = 1
    train_labels.append(one_hot)
train_labels = np.array(train_labels)
print(np.shape(train_labels))
# train_labels = train_labels.reshape((1, -1))

(2048, 40)


In [8]:
# one hot encode test_labels
test_labels_one_hot = []
for l in test_labels:
    one_hot = np.zeros(40, dtype=np.int)
    one_hot[l[0]] = 1
    test_labels_one_hot.append(one_hot)
test_labels_one_hot = np.array(test_labels_one_hot)
print(np.shape(test_labels_one_hot))

(2468, 40)


In [40]:
cloud = tf.placeholder(tf.float32, [None, 2048, 3])
pt_cloud = tf.expand_dims(cloud, -1)
print(np.shape(pt_cloud))

# placeholder for one-hot labels
y = tf.placeholder(tf.float32, [None, 40])

# placeholder for labels
y_labels = tf.placeholder(tf.int64, [None])

# 1st mlp layer
layer_conv1 = tf.contrib.layers.conv2d(inputs=pt_cloud, num_outputs=64, kernel_size=[1, 3], padding="VALID", activation_fn=tf.nn.relu)
print(np.shape(layer_conv1))

# 2nd mlp layer
layer_conv2 = tf.contrib.layers.conv2d(inputs=layer_conv1, num_outputs=64, kernel_size=[1, 1], activation_fn=tf.nn.relu)
print(np.shape(layer_conv2))

# 3rd mlp layer
layer_conv3 = tf.contrib.layers.conv2d(inputs=layer_conv2, num_outputs=64, kernel_size=[1, 1], activation_fn=tf.nn.relu)
print(np.shape(layer_conv3))

# 4th cnn
layer_conv4 = tf.contrib.layers.conv2d(inputs=layer_conv3, num_outputs=128, kernel_size=[1, 1], activation_fn=tf.nn.relu)
print(np.shape(layer_conv4))

# 5th cnn
layer_conv5 = tf.contrib.layers.conv2d(inputs=layer_conv4, num_outputs=1024, kernel_size=[1, 1], activation_fn=tf.nn.relu)
print(np.shape(layer_conv5))
# max pooling
max_pool = tf.contrib.layers.max_pool2d(inputs=layer_conv5, kernel_size=[2048, 1])

print(np.shape(max_pool))

# fnn1
layer_fnn1 = tf.contrib.layers.fully_connected(inputs=max_pool, num_outputs=512, activation_fn=tf.nn.relu)
print(np.shape(layer_fnn1))

# fnn2
layer_fnn2 = tf.contrib.layers.fully_connected(inputs=layer_fnn1, num_outputs=256, activation_fn=tf.nn.relu)
print(np.shape(layer_fnn2))

# fnn3
logits = tf.contrib.layers.fully_connected(inputs=layer_fnn2, num_outputs=40, activation_fn=tf.nn.relu)
logits = tf.squeeze(logits, [1, 2])
print(np.shape(logits))

# softmax
output = tf.nn.softmax(logits)
output_class = tf.argmax(output,axis=1)
print(np.shape(output))
print(np.shape(output_class))

(?, 2048, 3, 1)
(?, 2048, 1, 64)
(?, 2048, 1, 64)
(?, 2048, 1, 64)
(?, 2048, 1, 128)
(?, 2048, 1, 1024)
(?, 1, 1, 1024)
(?, 1, 1, 512)
(?, 1, 1, 256)
(?, 40)
(?, 40)
(?,)


In [63]:
# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

# optimizer
optim = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

In [65]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
num_iter = 1000
batch_size = 32

for i in range(num_iter):
    idx = np.random.choice(2048, [batch_size], False)
    batch_img = train_data[idx][:]
    batch_y = train_labels[idx][:]
#     batch_img, batch_y = data.train.next_batch(batch_size)
    _, l= sess.run([optim, loss], feed_dict = {cloud: batch_img , y: batch_y})
    if i % 10 == 0:
        print(l)

3.6883676
3.630666
3.6752417
3.4478877
3.5090272
3.1292477
3.3995001
2.842488
3.1004167
3.1292386
3.1057978
2.875028
2.5700054
3.0258102
2.5077255
2.6930163
2.7006013
2.946678
2.5633278
2.690843
3.1583443
2.6075232
2.3408334
2.7241614
2.4397483
2.8954577
2.7254045
2.2597213
2.1729813
3.0424538
2.5747154
2.242401
2.8485646
2.6325712
2.8062203
2.1256685
2.3185055
2.0054193
1.9384116
2.1578848
2.0010295
2.906476
2.746409
2.352211
3.0249243
3.0137627
2.7252495
2.7239833
2.354665
2.7254348
2.413688
2.3514051
2.5844688
1.8885489
2.9672565
2.7566178
2.050758
1.5851665
2.6654687
2.0931294
2.588356
1.9676259
2.4125078
2.165368
2.137185
1.879625
2.1571715
2.5405777
1.8131428
2.431563
1.9472493
2.172897
2.9700828
2.961849
2.290831
1.9131868
2.2979844
2.1478958
1.7348738
2.771872
2.592445
2.3853803
2.723796
1.9795232
2.375208
2.3821254
2.6270514
2.3427734
2.915947
1.5509845
1.9872304
1.7757483
1.5927855
1.973283
2.46439
2.228066
2.1079655
2.4159746
2.539044
2.007916


In [68]:
# accuracy
correct_labels = tf.equal(output_class, y_labels)
accuracy = tf.reduce_mean(tf.cast(correct_labels, tf.float32))
# compute accuracy on test data
labels = np.array([label.argmax() for label in test_labels_one_hot[:1000]])
#print(np.shape(labels))
accuracy = sess.run([accuracy],feed_dict = {cloud: test_data[:1000], y: test_labels_one_hot[:1000], y_labels: labels})
print ("Accuracy" ,accuracy)

(1000,)
Accuracy [0.405]


In [ ]:
# print(np.shape(test_labels_one_hot[:10]))
# labels = np.array([label.argmax() for label in test_labels_one_hot[:10]])
# print(np.shape(labels))
# print(np.shape(test_data[:10]))